# COVID -19 Interchange. Part II - Policy 

In [1]:
import pandas as pd

new_data = pd.read_csv(
    'https://beta.healthdata.gov/api/views/gyqz-9u7n/rows.csv?accessType=DOWNLOAD'
)

### Step 1. Data cleaning
After importing a full policy dataset into a dataframe, I need to extract only 'state' level policies and drop the column I will not need for this project

In [2]:
df = new_data[
    new_data.policy_level!='county'
].drop(columns=['county', 'fips_code', 'comments', 'source', 'total_phases'])

### Step 2. Data cleaning 
Here I am checking the data quality by checking the min and max date values as we could have some outliers that I would need to remove. Besides that, I change the data type of the column date and run the isnull to identify non datetime values I will have to remove. 

In [3]:
print(df.date.min())
print(df.date.max())

# if unable to change to datetime, it will be null
df['date'] = pd.to_datetime(
    df['date'],
    errors='coerce'
)

print(
    df.date.isnull().value_counts()
)

print(df.date.min())
print(df.date.max())

0020/03/25
2021/08/31
False    1669
True       33
Name: date, dtype: int64
2020-01-01 00:00:00
2021-08-31 00:00:00


Since more values are null (not datetime), it is easier to extract all the not null values that I will use and assign it to df dataframe.

In [4]:
df = df[df.date.notnull()]
df

,state_id,policy_level,date,policy_type,start_stop,geocoded_state
0,NE,state,2020-08-01,Phase 4,start,POINT (-99.810595 41.527099)
1,NV,state,2020-05-09,Non-Essential Businesses,start,POINT (-114.916992 39.283294)
3,AK,state,2020-05-22,Phase 3,start,POINT (-151.631889 63.631126)
4,SC,state,2020-05-04,Childcare (K-12),start,POINT (-80.864614 33.878572)
5,OR,state,2020-05-15,Entertainment,start,POINT (-120.605279 43.940449)
...,...,...,...,...,...,...
4202,AK,state,2020-04-24,Non-Essential Businesses,stop,POINT (-151.631889 63.631126)
4203,RI,state,2020-09-02,Mask Requirement,stop,POINT (-71.534637 41.572574)
4204,CO,state,2020-08-09,Phase 3,stop,POINT (-105.547832 38.998552)
4208,DC,state,2020-06-08,Shelter in Place,stop,POINT (-77.016275 38.904785)


### Step 3. Data cleaning. 
Here I check for date values that might not make sense (for example, 1989 would be inapropriate for covid dataframe)

In [5]:
mask = df.date.dt.year < 2019
display(df[mask])
df = df[
    df.date.dt.year > 2019
]

,state_id,policy_level,date,policy_type,start_stop,geocoded_state


### Step 4. Grouping categories of policy with a manually created csv table

In [6]:
# read new csv file with 2 columns 
policy_data = pd.read_csv('Policy_data.csv')

I can merge original policy data with a new category policy data to remap it. 

In [7]:
df = pd.merge(
    left=df,
    right=policy_data,
    left_on='policy_type',
    right_on='Current_Policy_Name',
    how='left'
)

### Step 5. Data cleaning
Here I remap the start_stop column into 1 and 0 instead of start and stop, accordingly. 

In [9]:
df['start_stop'] = df['start_stop'].map(dict(start = 1, stop = 0))

In [10]:
df.head(10)

,state_id,policy_level,date,policy_type,start_stop,geocoded_state,Current_Policy_Name,New_Policy_Category
0,NE,state,2020-08-01,Phase 4,1,POINT (-99.810595 41.527099),Phase 4,Phase 4
1,NV,state,2020-05-09,Non-Essential Businesses,1,POINT (-114.916992 39.283294),Non-Essential Businesses,Non essential business
2,AK,state,2020-05-22,Phase 3,1,POINT (-151.631889 63.631126),Phase 3,Phase 3
3,SC,state,2020-05-04,Childcare (K-12),1,POINT (-80.864614 33.878572),Childcare (K-12),Childcare/Daycare
4,OR,state,2020-05-15,Entertainment,1,POINT (-120.605279 43.940449),Entertainment,Entertainment
5,OR,state,2020-05-15,Entertainment,1,POINT (-120.605279 43.940449),Entertainment,Entertainment
6,KY,state,2020-08-25,Stop Enforcement Of Evictions Overall Or Due T...,1,POINT (-85.290484 37.526683),Stop Enforcement Of Evictions Overall Or Due T...,Household assistance
7,KY,state,2020-06-01,Outdoor and Recreation,1,POINT (-85.290484 37.526683),Outdoor and Recreation,Outdoor and Recreation
8,GA,state,2020-08-16,Phase 2,1,POINT (-83.426574 32.638481),Phase 2,Phase 2
9,WA,state,2020-07-20,Phase 3,1,POINT (-120.592492 47.411715),Phase 3,Phase 3


In [11]:
# rename column state_id into state to match another dataset
df.rename(columns = {'state_id':'state'}, inplace = True)

### Step 6. Data cleaning
Checking for duplicate rows in the dataframe. I am choosing to drop all the duplicate rows as they are not needed. 

In [12]:
df[df.duplicated()] #['state_id', 'clean_date', 'New_Policy_Category']

,state,policy_level,date,policy_type,start_stop,geocoded_state,Current_Policy_Name,New_Policy_Category
5,OR,state,2020-05-15,Entertainment,1,POINT (-120.605279 43.940449),Entertainment,Entertainment
170,IA,state,2020-06-25,Food and Drink,0,POINT (-93.500061 42.074659),Food and Drink,Dining out restrictions-food
186,MT,state,2020-06-01,New Phase,0,POINT (-109.645127 47.033521),New Phase,New phase
191,NE,state,2020-12-31,Mask Requirement,0,POINT (-99.810595 41.527099),Mask Requirement,Mask mandate
237,CO,state,2020-07-20,Executive Order,0,POINT (-105.547832 38.998552),Executive Order,State of emergency/quarantine
...,...,...,...,...,...,...,...,...
1588,NC,state,2020-10-02,Entertainment,0,POINT (-79.130883 35.52603),Entertainment,Entertainment
1607,WY,state,2020-08-31,Phase 3,0,POINT (-107.55145 42.999627),Phase 3,Phase 3
1608,MT,state,2020-06-01,New Phase,0,POINT (-109.645127 47.033521),New Phase,New phase
1617,MS,state,2020-08-31,Mask Requirement,0,POINT (-89.65344 32.713378),Mask Requirement,Mask mandate


In [13]:
# dropping all the duplicates
df_dropped = df.drop_duplicates()

In [14]:
# after dropping the 80 duplicate rows I have 1579 rows left
len(df_dropped)

1586

### Step 7. Changing the policy DataFrame with PivotTable
To do the task I am using 'state', 'date' and new_policy_category columns and fill it up with start_stop column values. 

In [16]:
df_policy = pd.pivot_table(df_dropped, index=['state','date'], columns = ['New_Policy_Category'], values = 'start_stop') #, fill_value = ''
df_policy = df_policy.sort_values(by=['state', 'date'])

In [17]:
df_policy = df_policy.reset_index()

In [18]:
df_policy

New_Policy_Category,state,date,Childcare/Daycare,Dining out restrictions-food,Education,Election,Entertainment,Essencial,Gyms,Household assistance,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
0,AK,2020-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,AK,2020-03-16,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,2020-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,2020-03-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,2020-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,WY,2020-06-15,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,WY,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
856,WY,2020-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,WY,2020-07-31,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
import datetime as dt

today_date = dt.datetime.now().date()

dates = []

for state_name, state_subset in df_policy.groupby('state'):
    dates.append(pd.DataFrame(data={
        'state': state_name,
        'date':pd.date_range(
#             subset.date.min() - dt.timedelta(days = 7)
            state_subset['date'].min(),
            today_date
#             subset.date.max() + dt.timedelta(days = 7)
        )}
))

In [21]:
active_dates = pd.concat(dates)
#active_dates = active_dates.drop_duplicates()
#print(len(active_dates.index))
active_dates

,state,date
0,AK,2020-03-11
1,AK,2020-03-12
2,AK,2020-03-13
3,AK,2020-03-14
4,AK,2020-03-15
...,...,...
418,WY,2021-05-05
419,WY,2021-05-06
420,WY,2021-05-07
421,WY,2021-05-08


In [22]:
temp = active_dates.merge(
    df_policy,
    on=['state', 'date'],
    how='outer')#.ffill()

print(temp.shape)
temp

(23779, 37)


,state,date,Childcare/Daycare,Dining out restrictions-food,Education,Election,Entertainment,Essencial,Gyms,Household assistance,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
0,AK,2020-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,AK,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,2020-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,2020-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,2020-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23774,ME,2021-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23775,MI,2021-05-24,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23776,MI,2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23777,MO,2021-08-31,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
active_dates['date'] = pd.to_datetime(
    active_dates['date'],
    errors='coerce'
)

In [24]:
temp.set_index('state', inplace=True)

### Step 8. Forward fill and fill missing values with 1 and 0. 
Here I am renaming 1.0 and 0.0 into 'In effect' and 'not in effect', so all the rows are filled with values that makes sense for the dataframe. 

In [25]:
# forward fill with group by state
temp = temp.groupby('state').ffill().fillna(0.0)
temp

,date,Childcare/Daycare,Dining out restrictions-food,Education,Election,Entertainment,Essencial,Gyms,Household assistance,Houses of Worship,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
state,,,,,,,,,,,,,,,,,,,,,
AK,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AK,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AK,2020-03-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AK,2020-03-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AK,2020-03-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ME,2021-05-13,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
MI,2021-05-24,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
MI,2021-05-31,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Step 9. Final Policy data frame 

In [26]:
temp = temp.replace(0.0, 'not in effect').replace(1.0, 'in effect')
temp

,date,Childcare/Daycare,Dining out restrictions-food,Education,Election,Entertainment,Essencial,Gyms,Household assistance,Houses of Worship,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
state,,,,,,,,,,,,,,,,,,,,,
AK,2020-03-11,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,...,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2020-03-12,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,...,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2020-03-13,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,...,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2020-03-14,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,...,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2020-03-15,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,...,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ME,2021-05-13,in effect,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,in effect,not in effect,...,not in effect,in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
MI,2021-05-24,not in effect,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,in effect,in effect,...,not in effect,in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect
MI,2021-05-31,not in effect,not in effect,not in effect,not in effect,not in effect,in effect,not in effect,in effect,in effect,...,not in effect,in effect,not in effect,not in effect,in effect,not in effect,not in effect,not in effect,not in effect,not in effect


### Step 10. Working with previous data frame

In [27]:
import pandas as pd
# importing previous dataset and setting an index column 'state'
previous = pd.read_csv('final_covid.csv', index_col = 1)
previous.head(3)

,date,fips,cases,deaths,Inpatient Beds Occupied Estimated,Count LL_inpatient beds occupied,Count UL_inpatient beds occupied,Percentage of Inpatient Beds Occupied Estimated,Percentage LL_inpatient beds occupied,Percentage UL_inpatient beds occupied,...,Staffed Adult ICU Beds Occupied Estimated,Count LL_ICU Beds Occupied,Count UL_ICU Beds Occupied,Percentage of Staffed Adult ICU Beds Occupied Estimated,Percentage LL_ICU Beds Occupied,Percentage UL_ICU Beds Occupied,Total Staffed Adult ICU Beds,Total LL_ICU Beds Occupied,Total UL_ICU Beds Occupied,geocoded_state
state,,,,,,,,,,,,,,,,,,,,,
AL,2021-04-07,1,517452,10652,10285,10285,10285,73.95,73.95,73.95,...,1332,1332,1332,84.09,84.09,84.09,1584,1584,1584,POINT (-86.844516 32.756889)
AK,2021-04-07,2,63785,299,886,886,886,59.54,59.54,59.54,...,90,90,90,63.83,63.83,63.83,141,141,141,POINT (-151.631889 63.631126)
AZ,2021-04-07,4,846241,17025,9891,9889,9893,69.79,69.68,69.90,...,1062,1062,1062,49.05,49.05,49.05,2165,2165,2165,POINT (-111.664616 34.293239)


In [28]:
# changing datatype of column 'date'
previous['date'] = pd.to_datetime(previous['date'], errors='coerce')

### Step 11. Merging two DataFrames

In [29]:
mrg_on_state = pd.merge(previous, df_policy, how = "left", on = ['date', 'state'])
mrg_on_state.sort_values(by=['state','date'])

,date,state,fips,cases,deaths,Inpatient Beds Occupied Estimated,Count LL_inpatient beds occupied,Count UL_inpatient beds occupied,Percentage of Inpatient Beds Occupied Estimated,Percentage LL_inpatient beds occupied,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
1,2021-04-07,AK,2,63785,299,886,886,886,59.54,59.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,2021-04-08,AK,2,64013,299,860,860,860,57.95,57.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,2021-04-09,AK,2,64259,299,837,828,846,56.48,56.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,2021-04-10,AK,2,64259,299,830,829,831,56.12,56.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,2021-04-11,AK,2,64259,299,797,797,797,53.89,53.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,2021-05-03,WY,56,58290,707,492,488,496,34.26,31.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1427,2021-05-04,WY,56,58367,710,622,619,626,43.35,43.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,2021-05-05,WY,56,58438,710,616,550,683,42.93,25.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1529,2021-05-06,WY,56,58527,710,626,540,712,43.59,21.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
mrg_on_state.set_index('state', inplace=True)

mrg_on_state

,date,fips,cases,deaths,Inpatient Beds Occupied Estimated,Count LL_inpatient beds occupied,Count UL_inpatient beds occupied,Percentage of Inpatient Beds Occupied Estimated,Percentage LL_inpatient beds occupied,Percentage UL_inpatient beds occupied,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
state,,,,,,,,,,,,,,,,,,,,,
AL,2021-04-07,1,517452,10652,10285,10285,10285,73.95,73.95,73.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2021-04-07,2,63785,299,886,886,886,59.54,59.54,59.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,2021-04-07,4,846241,17025,9891,9889,9893,69.79,69.68,69.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR,2021-04-07,5,331505,5660,5370,5370,5370,69.70,69.70,69.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,2021-04-07,6,3687635,59975,43622,43622,43622,73.89,73.89,73.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VA,2021-05-07,51,665332,10874,11663,11432,11894,72.82,68.26,77.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WA,2021-05-07,53,414479,5609,8927,8270,9584,74.22,56.86,91.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WV,2021-05-07,54,155680,2719,3887,3887,3887,77.93,77.93,77.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#x = [0, 1] 2-32, 33-67

mrg_on_state.iloc[:, 2:32] = mrg_on_state.iloc[:, 2:32].fillna(' ')
mrg_on_state.sort_values(by=['state','date'])

,date,fips,cases,deaths,Inpatient Beds Occupied Estimated,Count LL_inpatient beds occupied,Count UL_inpatient beds occupied,Percentage of Inpatient Beds Occupied Estimated,Percentage LL_inpatient beds occupied,Percentage UL_inpatient beds occupied,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
state,,,,,,,,,,,,,,,,,,,,,
AK,2021-04-07,2,63785,299,886,886,886,59.54,59.54,59.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2021-04-08,2,64013,299,860,860,860,57.95,57.95,57.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2021-04-09,2,64259,299,837,828,846,56.48,56.48,56.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2021-04-10,2,64259,299,830,829,831,56.12,56.12,56.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,2021-04-11,2,64259,299,797,797,797,53.89,53.89,53.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,2021-05-03,56,58290,707,492,488,496,34.26,31.92,36.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WY,2021-05-04,56,58367,710,622,619,626,43.35,43.35,43.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WY,2021-05-05,56,58438,710,616,550,683,42.93,25.60,60.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#mrg_on_state.iloc[:, x].groupby('state').ffill() #.fillna(0)
mrg_on_state = mrg_on_state.groupby('state').ffill().fillna('not in effect')

In [33]:
mrg_on_state.sort_values(by=['state','date'])

,date,fips,cases,deaths,Inpatient Beds Occupied Estimated,Count LL_inpatient beds occupied,Count UL_inpatient beds occupied,Percentage of Inpatient Beds Occupied Estimated,Percentage LL_inpatient beds occupied,Percentage UL_inpatient beds occupied,...,Quarantine,Remote Healthcare/postpone elective procedures,Residential Overnight Camps,Shelter in Place,State of Emergency,State of Emergency/Funds,State of emergency/quarantine,Training jobs,Travel,Updated Guidelines
state,,,,,,,,,,,,,,,,,,,,,
AK,2021-04-07,2,63785,299,886,886,886,59.54,59.54,59.54,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2021-04-08,2,64013,299,860,860,860,57.95,57.95,57.95,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2021-04-09,2,64259,299,837,828,846,56.48,56.48,56.48,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2021-04-10,2,64259,299,830,829,831,56.12,56.12,56.12,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
AK,2021-04-11,2,64259,299,797,797,797,53.89,53.89,53.89,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,2021-05-03,56,58290,707,492,488,496,34.26,31.92,36.61,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
WY,2021-05-04,56,58367,710,622,619,626,43.35,43.35,43.35,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect
WY,2021-05-05,56,58438,710,616,550,683,42.93,25.60,60.26,...,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect,not in effect


### Save to the file

In [ ]:
#mrg_on_state.to_csv('COVID_policy_left.csv')

In [ ]:
#temp.to_csv('policy_final.csv')